In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

def main(sirfile1,sirfile2,centfile1,centfile2,degree_cent):
    label=[2,5,10,20,30,40,50,60,70,80,90,100]
    fin_dict={}
    fin_prec={}
    fin_rec={}
    fin_f1={}
    for z in label:
        df=pd.read_csv(sirfile1)
        df['SIR_IC']=(df['infSIR']+df['infIC'])/2
        df1=df.drop(['infSIR','infIC','beta'],axis=1)
        df1.rename(columns = {'vertex':'Node'}, inplace = True)
        df2 = pd.read_csv(centfile1)
        df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
        def labelling1(x,N):
            minscale=df2[degree_cent].min()
            maxscale=df2[degree_cent].max()
            rang=(maxscale-minscale)/N
            label=math.floor((x-minscale)//rang)+1
            return label
        df2['Label'] =df2.apply(lambda row:labelling1(row[degree_cent],z),axis=1)
        df4=pd.merge(df1,df2)
        l=dict(df4['Label'].value_counts())
        s = {k : l[k] for k in sorted(l,reverse=True)}
        x1=[]
        k=list(s.keys())
        sum1=0
        for i in k:
            sum1 = sum1+s[i]
            x=sum1/sum(s.values())
            x1.append(x)
        x2=map(lambda x:x*1000,x1)
        x3=[]
        for i in x2:
            x3.append(i)
        x3=x3[::-1]
        x3.append(0.0)
        x3[0]=1001.0
        x4=[]
        for x in x3:
            x4.append(round(x,2))
        res = list(zip(x4, x4[1:]))
        dict_res={}
        k=1
        for i in res:
            dict_res[k]=list(i)
            k+=1
        lis=[]
        for j in dict_res.values():
            x=list(np.arange(j[1],j[0],0.01))
            lis.append(x)
        lis_2=[]
        for i in lis:
            new_list = list(np.around(np.array(i), 2))
            lis_2.append(new_list)
        def pred_label(rank):
            lab=0
            for l in lis_2:
                if rank in l:
                    lab=(lis_2.index(l))+1
            return lab
        def labelling2(x,N):
            minscale=df2_new[degree_cent].min()
            maxscale=df2_new[degree_cent].max()
            rang=(maxscale-minscale)/N
            label=math.floor((x-minscale)//rang)+1
            return label
        df_new=pd.read_csv(sirfile2)
        df_new['SIR_IC']=(df_new['infSIR']+df_new['infIC'])/2
        df1_new=df_new.drop(['infSIR','infIC','beta'],axis=1)
        df1_new.rename(columns = {'vertex':'Node'}, inplace = True)
        df2_new = pd.read_csv(centfile2)
        df2_new.drop(df2_new.columns.difference(['Node',degree_cent]), 1, inplace=True)
        df2_new['Label'] =df2_new.apply(lambda row:labelling2(row[degree_cent],z),axis=1)
        df4_new=pd.merge(df1_new,df2_new)
        df5=df4_new.sort_values('SIR_IC',ascending=False)
        l=[]
        for i in range(1,len(df5)+1):
            l.append(i)
        df5['Rank']=l
        df5['Rank']=df5['Rank']/len(df5)
        l1=list(df5['Rank'])
        l2=map(lambda x:x*1000,l1)
        l3=[]
        for i in l2:
            l3.append(round(i,2))
        df5=df5.drop(['Rank'],axis=1)
        df5['Rank']=l3
        df5['Predicted_Label'] =df5.apply(lambda row:pred_label(row['Rank']),axis=1)
        y_pred = list(df5['Predicted_Label'])
        y_true = list(df5['Label'])
        fin_res=accuracy_score(y_true, y_pred)
        prec=precision_score(y_true,y_pred,average="macro")
        rec=recall_score(y_true,y_pred,average="macro")
        f1=f1_score(y_true,y_pred,average="macro")
        fin_dict[z]=fin_res
        fin_prec[z]=prec
        fin_rec[z]=rec
        fin_f1[z]=f1
    return fin_dict,fin_prec,fin_rec,fin_f1




l1=['DC','B','C','CC','PR','KC','LRAD','GRAD','LRCC','GRACC','LRKC','GRAK']

for i in range(0,len(l1)):
    d1,d2,d3,d4=main('siricforall(web_0.2).csv','siricforall(USAir97_0.2).csv','centraltiesallverticetest_web(21-03-2023).csv','centraltiesallverticetest_USAir97.csv',l1[i])
    new_list1 = list(d1.keys())
    new_list2 = list(d1.values())
    new_list3=list(d2.values())
    new_list4=list(d3.values())
    new_list5=list(d4.values())
    df=pd.DataFrame(list(zip(new_list1,new_list2,new_list3,new_list4,new_list5)),columns=['Labels','Accuracy','Precision','Recall','F1-score'])
    df.to_csv('{}.csv'.format(l1[i]))

C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2_new.drop(df2_new.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of 

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

...

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all a

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2.drop(df2.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\AppData\Local\Temp\ipykernel_21440\3956445278.py:78: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df2_new.drop(df2_new.columns.difference(['Node',degree_cent]), 1, inplace=True)
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defin

C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bhargavi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
